<a href="https://colab.research.google.com/github/hopecris31/Reddit_Transformer/blob/master/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[sentencepiece]
!pip install datasets
!pip install evaluate

In [2]:
import transformers
import math
from google.colab import drive
from transformers import AutoTokenizer, AutoModelForCausalLM

In [3]:
checkpoint = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, use_fast=True, max_length=10)
model = AutoModelForCausalLM.from_pretrained(checkpoint)


In [4]:
from datasets import load_dataset

In [5]:
raw_datasets = load_dataset("reddit", split="train[:0.05%]")
raw_datasets

Generating train split:   0%|          | 0/3848330 [00:00<?, ? examples/s]

Dataset reddit downloaded and prepared to /root/.cache/huggingface/datasets/reddit/default/1.0.0/98ba5abea674d3178f7588aa6518a5510dc0c6fa8176d9653a3546d5afcb3969. Subsequent calls will reuse this data.


Dataset({
    features: ['author', 'body', 'normalizedBody', 'subreddit', 'subreddit_id', 'id', 'content', 'summary'],
    num_rows: 192416
})

In [6]:
print(raw_datasets.column_names)

['author', 'body', 'normalizedBody', 'subreddit', 'subreddit_id', 'id', 'content', 'summary']


In [7]:
def tokenize_function(example):
    return tokenizer(example["subreddit"], example["content"])
    

In [8]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=raw_datasets.column_names)
tokenized_datasets

Map (num_proc=4):   0%|          | 0/192416 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2066 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1305 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2287 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1098 > 1024). Running this sequence through the model will result in indexing errors


Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 192416
})

In [9]:

tokenized_datasets[1]


{'input_ids': [12543,
  3281,
  8001,
  318,
  546,
  262,
  17612,
  1517,
  284,
  17851,
  1096,
  287,
  2846,
  286,
  922,
  290,
  2089,
  13,
  1675,
  2074,
  530,
  670,
  393,
  6802,
  355,
  17863,
  625,
  1194,
  2058,
  866,
  284,
  2614,
  4459,
  13,
  10889,
  617,
  1243,
  3863,
  39340,
  1365,
  621,
  584,
  2499,
  11,
  475,
  340,
  6165,
  7363,
  351,
  262,
  1981,
  13,
  314,
  7620,
  2883,
  262,
  670,
  286,
  366,
  25662,
  7912,
  1,
  357,
  3500,
  50056,
  407,
  284,
  307,
  47037,
  11,
  475,
  8384,
  780,
  428,
  318,
  287,
  257,
  1180,
  6536,
  621,
  28530,
  290,
  1201,
  616,
  4469,
  318,
  407,
  287,
  1242,
  314,
  836,
  470,
  760,
  644,
  262,
  366,
  1676,
  525,
  1,
  3381,
  318,
  837,
  611,
  612,
  318,
  530,
  828,
  475,
  314,
  466,
  766,
  810,
  345,
  389,
  2406,
  422,
  13,
  327,
  28882,
  12444,
  284,
  779,
  262,
  976,
  4263,
  17282,
  357,
  1462,
  257,
  966,
  810,
  530,
  714,
  910

In [10]:
block_size = tokenizer.model_max_length

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum([ex for ex in examples[k] if isinstance(ex, list)], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [11]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/192416 [00:00<?, ? examples/s]

In [12]:
lm_datasets

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 48275
})

In [13]:
tokenizer.decode(lm_datasets[1]["input_ids"])

' progress like you are now, fighting the hardest monsters you can. When a potential good white item appears, try to imbue that. Class specifiq items has a better chance to give +skills to your class. Circlets has the higest bonuses regarding yellow items. And you can get an extra base stat advantage by using exceoptional (nightmare) items, which will drop in a4-5 from time to time. You can check out monster levels, and item qualities (normal, exceptional, elite) on arreat summit ofc. :) \n EDIT (forgot infoz):\nIn cases of imbue-eligible items with base bonuses (maces, wands, staffs and class specifique items, and any other item with +skills etc.) will loose their current bonuses due to the base item reroll. They can get extra skills from the base item AND the rarity class. Say, if you imbue a staff with +1 fireball, it will reroll that base staff. It might turn out with a staff with +1 icebolt and +1 warmth, and then you add the bonuses the rarity would give you, lets say +1 icebolt 

In [14]:
from transformers import TrainingArguments, Trainer

In [15]:
model_name = checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-reddit",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False
)

In [19]:
from google.colab import drive
drive.mount('/content/drive')
PATH = 'Drive/NLP Final' # [CHANGE ME]

Mounted at /content/drive


In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets,
    eval_dataset=lm_datasets,
)

In [17]:
trainer.train()
drive.mount('/content/drive')
PATH = 'Drive/NLP Final' # [CHANGE ME]
trainer.save_model (PATH + "/ChatGRT")

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 48275
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 18105
  Number of trainable parameters = 81912576


Epoch,Training Loss,Validation Loss
1,3.801900,3.675585
2,3.761900,3.649620
3,3.748900,3.640436


Saving model checkpoint to distilgpt2-finetuned-reddit/checkpoint-500
Configuration saved in distilgpt2-finetuned-reddit/checkpoint-500/config.json
Configuration saved in distilgpt2-finetuned-reddit/checkpoint-500/generation_config.json
Model weights saved in distilgpt2-finetuned-reddit/checkpoint-500/pytorch_model.bin
Saving model checkpoint to distilgpt2-finetuned-reddit/checkpoint-1000
Configuration saved in distilgpt2-finetuned-reddit/checkpoint-1000/config.json
Configuration saved in distilgpt2-finetuned-reddit/checkpoint-1000/generation_config.json
Model weights saved in distilgpt2-finetuned-reddit/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to distilgpt2-finetuned-reddit/checkpoint-1500
Configuration saved in distilgpt2-finetuned-reddit/checkpoint-1500/config.json
Configuration saved in distilgpt2-finetuned-reddit/checkpoint-1500/generation_config.json
Model weights saved in distilgpt2-finetuned-reddit/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to di

TrainOutput(global_step=18105, training_loss=3.782132766247454, metrics={'train_runtime': 9183.5432, 'train_samples_per_second': 15.77, 'train_steps_per_second': 1.971, 'total_flos': 3.78423019044864e+16, 'train_loss': 3.782132766247454, 'epoch': 3.0})

In [18]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 48275
  Batch size = 8


Perplexity: 38.11
